In [31]:
import spacy
import pandas as pd
import re

# Cargar el DataFrame
data = pd.read_csv(r'F:\ManuelAngulo\PROYECTOS\ANALISIS DE SENTIMIENTOS\data\X_dumek_data.csv')

# Eliminar filas con valores nulos en todas las columnas (opcional)
data.dropna(axis=0, inplace=True)

In [32]:
# Definir los rangos no continuos y el delimitador para cada uno
ranges = {
    (0, 2): 'https://',
    (152, 192): 'http://',
    (546, 567): 'http://',
    (760, 769): '#cartagena',
    (835, 901): '...',  # Ejemplo de rangos; usa los rangos que necesites
    (1230, 1279): '? ',
    (1548, 1555): 'este Fin de Semana en Cartagena.',
}

# Función para aplicar la separación de texto y comentarios
def split_text(text, delimiter):
    parts = text.split(delimiter, 1)  # Solo divide en la primera aparición del delimitador
    post_text = parts[0].strip() if parts else text
    comments = delimiter + parts[1].strip() if len(parts) > 1 else ''
    return post_text, comments

# Lista para guardar nuevas filas
new_rows = []

# Aplicar la separación a los rangos específicos
for (start, end), delimiter in ranges.items():
    for idx in range(start, end + 1):
        if idx < len(data):  # Verifica si el índice está dentro del rango del DataFrame
            post_text, new_comments = split_text(data.at[idx, 'Tweet Texto'], delimiter)
            
            # Actualizar la fila original con el texto del post
            data.at[idx, 'Tweet Texto'] = post_text
            
            # Añadir nueva fila para los comentarios extraídos
            if new_comments:
                new_row = {
                    'Tweet ID': data.at[idx, 'Tweet ID'],
                    'Fecha de Publicación': data.at[idx, 'Fecha de Publicación'],
                    'Tweet Texto': post_text,  # Vacío porque el texto del post ya está en la fila original
                    'Comentario': new_comments
                }
                new_rows.append(new_row)

# Convertir la lista de nuevas filas a un DataFrame
new_df = pd.DataFrame(new_rows)

# Concatenar el DataFrame original con el nuevo DataFrame
data = pd.concat([data, new_df], ignore_index=True)
data = data.drop_duplicates().reset_index(drop=True)  # Opcional: eliminar duplicados y reiniciar el índice

In [33]:
# Procesar los comentarios para segmentarlos
def segmentar_texto(text_list):
    segmented_comments = []
    current_comment = ""
    
    for text in text_list:
        # Divide el texto en líneas basadas en saltos de línea
        comentarios = text.split('\n')
        comentarios = [comentario.strip() for comentario in comentarios if comentario.strip()]
        
        for comentario in comentarios:
            # Si el comentario empieza con una letra minúscula o con signos de puntuación, lo une al comentario actual
            if comentario and (comentario[0].islower() or comentario[0] in ',.!?'):
                if current_comment:
                    current_comment += " " + comentario
            else:
                # Si hay un comentario acumulado, lo añade a la lista
                if current_comment:
                    segmented_comments.append(current_comment)
                # Comienza un nuevo comentario si el comentario empieza con una letra mayúscula
                if comentario and comentario[0].isupper():
                    current_comment = comentario
                else:
                    current_comment = ""
    
    # Agrega el último comentario si existe
    if current_comment:
        segmented_comments.append(current_comment)
    
    return segmented_comments

# Aplicar la función de segmentación de texto a cada comentario
data['Comentario'] = data['Comentario'].apply(lambda x: x.split('\n') if isinstance(x, str) else x)
data['Comentario'] = data['Comentario'].apply(segmentar_texto)

# Desplegar listas en filas individuales
data = data.explode('Comentario')
data = data[data['Comentario'] != 'Dumek Turbay']
data = data.reset_index(drop=True)

In [34]:
# Cargar el modelo de spaCy para español
nlp = spacy.load('es_core_news_md')

# Definir una lista de palabras que se desean eliminar
custom_stopwords = set([
    "y", "o", "de", "la", "el", "en", "un", "una", "los", "las",
    "por", "con", "para", "es", "son", "del", "se", "al", 'x', 'este', 'esta'
])

def custom_filter(text):
    # Procesar el texto con spaCy
    doc = nlp(text)
    
    # Eliminar palabras de la lista personalizada y puntuación, pero mantener los espacios
    filtered_tokens = [
        token.text for token in doc if not token.is_punct and token.text.lower() not in custom_stopwords
    ]
    
    # Unir los tokens en un solo string con espacios, eliminando números y caracteres especiales adicionales
    filtered_text = " ".join(filtered_tokens)
    filtered_text = re.sub(r'\b\d+\b', '', filtered_text)  # Elimina números
    filtered_text = re.sub(r'[^A-Za-zÀ-ÿñÑ ]+', '', filtered_text)  # Elimina caracteres no alfanuméricos y no latinos

    return filtered_text.strip()

# Filtrar palabras basura
palabras_basura = ['Tigo', 'Claro', 'Juegos Olímpicos', 'Maduro', 'Trading', 'trader','traders','trade', 'trades', 'VENEZUELA', 'Postpago', 'Venezuela', 'apuestas', 'Cc', 
                   'MVM', 'leaks', 'generator', 'learn', 'NEREDE', 'quality', 'takes', 'Coding', 'Más detalles aquí', 'Argos', 'Q', 'Transmisión' ]
regex = '|'.join([f'\\b{re.escape(palabra)}\\b' for palabra in palabras_basura])
data = data[~data['Comentario'].str.contains(regex, case=False, na=False)]

# Limpiar los comentarios usando la función custom_filter
data['Comentario Limpio'] = data['Comentario'].apply(custom_filter)
data['Tweet Texto Limpio'] = data['Tweet Texto'].apply(custom_filter)
# Filtrar filas vacías en 'Comentario Limpio'
data = data[data['Comentario Limpio'].str.strip() != '']


In [35]:
# Eliminar las filas con índices 0 y 1
data.drop([0, 1, 3, 303, 982, 1085, 1295, 1297, 1513], axis=0, inplace=True)
data['Comentario Limpio'] = data['Comentario Limpio'].str.lower()
comentario_basura = ['desde','alcalde', 'sr alcalde', 'estimado alcalde','si', 'ajá', 'señor alcalde', 'nota'] 
data = data[~data['Comentario Limpio'].isin(comentario_basura)]

In [36]:
data.reset_index()

,index,Tweet ID,Fecha de Publicación,Tweet Texto,Comentario,Comentario Limpio,Tweet Texto Limpio
0,2,1810294804602896601,2024-07-08,Señores dueños de vehículos de carga pesada: N...,HOY se abren las puertas de la monumental,hoy abren puertas monumental,Señores dueños vehículos carga pesada Ni Martí...
1,4,1810326741518315926,2024-07-08,"Miércoles 10 de julio \n7:00\n p.m., nuestra \...","Señor Alcalde, el sábado juega la selección Co...",señor alcalde sábado juega selección colombia ...,Miércoles julio pm nuestra FCFSeleccionC...
2,5,1810326741518315926,2024-07-08,"Miércoles 10 de julio \n7:00\n p.m., nuestra \...",Invasores ilegales hasta la orilla en Playa bl...,invasores ilegales hasta orilla playa blanca b...,Miércoles julio pm nuestra FCFSeleccionC...
3,6,1810326741518315926,2024-07-08,"Miércoles 10 de julio \n7:00\n p.m., nuestra \...","DIMAR ALCALDÍA,MEK &MAR",dimar alcaldía mek mar,Miércoles julio pm nuestra FCFSeleccionC...
4,7,1810326741518315926,2024-07-08,"Miércoles 10 de julio \n7:00\n p.m., nuestra \...",Mi alcalde necesito de todo su apoyo para subi...,mi alcalde necesito todo su apoyo subir bander...,Miércoles julio pm nuestra FCFSeleccionC...
...,...,...,...,...,...,...,...
1473,1571,1815883010773758151,2024-07-23,¿Y por qué no una APP donde el usuario conzoca...,Tal cual es lo que hacen las plataformas de te...,tal cual lo que hacen plataformas tecnología,qué no APP donde usuario conzoca tarifa antemano
1474,1572,1816855553143980067,2024-07-26,HOY es la gran reinauguración del Parque Espír...,Cuantos años tiene el espíritu del manglar ? ...,cuantos años tiene espíritu manglar parque ...,HOY gran reinauguración Parque Espíritu Mangla...
1475,1573,1816855553143980067,2024-07-26,HOY es la gran reinauguración del Parque Espír...,Excelente plan pra visitar este fin de semana....,excelente plan pra visitar fin semana alcalde ...,HOY gran reinauguración Parque Espíritu Mangla...
1476,1574,1816855553143980067,2024-07-26,HOY es la gran reinauguración del Parque Espír...,Todos a visitar este parque manglar. A maravil...,todos a visitar parque manglar a maravillarnos...,HOY gran reinauguración Parque Espíritu Mangla...


In [37]:
temas_generales = { 
    1810294804602896601: ('Movilidad','Vehículos de carga'),
    1810326741518315926: ('Fútbol','Pantallas gigantes'),
    1810457618747449505: ('Ayuda social','Alexy Hernandez'),
    1810683396521156736: ('Ayuda social','Hogar de paso'),
    1810774488868605956: ('Movilidad','Corredor de carga'),
    1810840233451343925: ('Afinia','Salud'),
    1811110191058014270: ('Feminicidio','Emperatriz Alquerque'),
    1811570705282036150: ('Feminicidio','Emperatriz Alquerque'),
    1811760830305714449: ('Movilidad','Mamonal'),
    1811779138262303176: ('Educación','Infraestructura'),
    1811889122023473381: ('Fútbol','Día civico'),
    1812186968954180047: ('Fútbol','Seguridad'),
    1812272863937782081: ('Fútbol','Selección femenina'),
    1812651169199665584: ('Fútbol','Pantallas gigantes'),
    1812467567744745721: ('Infraestructura','Centro histórico'),
    1812508352175624619: ('Educación','Día civico'),
    1812831308155892145: ('Alcaldía','Cooperación internacional'),     
    1812866188638044619: ('Salvavidas','Cooperación internacional'),
    1812872308698034284: ('Alcaldía','Cooperación internacional'),
    1812898373692785101: ('Lucha contra violencia de genero','Atención a victimas'),
    1813431961685197095: ('Movilidad','Transcaribe'),
    1813231644456665589: ('Ayuda social','Alexy Hernandez'),
    1813198073415311448: ('Educación','Infraestructura'),
    1813587996974940614: ('Ayuda social','Alexy Hernandez'),
    1813658549777764553: ('Financiación de proyectos','Cooperación internacional'),
    1813729204661612670: ('Alcaldía','Cooperación internacional'),
    1813907616600310153: ('Movilidad','Infraestructura'),
    1813954949648208068: ('Lucha contra trata de personas','Cooperación internacional'),
    1814057186357891290: ('Financiación de proyectos','Cooperación internacional'),
    1814324157351420362: ('Financiación de proyectos','Cooperación internacional'),
    1814332426178424863: ('Infraestructura','Centro histórico'),
    1814337751870845364: ('Eventos internacionales','Cooperación internacional'),
    1814431657887445067: ('Feminicidio','Emperatriz Alquerque'),
    1814717384638304271: ('Fechas patrías','20 de julio'),
    1814750646186242227: ('Movilidad','Infraestructura'),
    1815017002123079726: ('Alcaldía','Gestión'),  
    1815059716390682778: ('Infraestructura','Plaza de toros'),
    1815064534010261963: ('Infraestructura','Parque espíritu del manglar'),
    1815103545924112612: ('Infraestructura','Canal Matute'),
    1815355396170690747: ('Propuestas de campaña','Comisión Fílmica'),
    1815743840428405063: ('Movilidad','Taxistas'),
    1815755918904324104: ('Movilidad','Taxistas'),
    1815877187792429496: ('Movilidad','Taxistas'),
    1815878502823923876: ('Fútbol','Cancha de los calamares'),
    1815883010773758151: ('Movilidad','Taxistas'),
    1816153177894645931: ('Movilidad','Infraestructura'),
    1816155552139076061: ('Infraestructura','Planificación urbana'),
    1816212915554488646: ('Salud','Abrazo de Dumek'),
    1816336055303299122: ('Movilidad','Barrio Crespo'),
    1816498777722700073: ('Movilidad','Infraestructura'),
    1816901513630888404: ('Movilidad','Mamonal'),
    1816894816166302179: ('Infraestructura','Parque de la marina'),
    1816862407999779122: ('Explotación sexual','Alianzas institucionales'),
    1816859264540954696: ('Infraestructura','Parque espíritu del manglar'),
    1816856544002085065: ('Infraestructura','Parque espíritu del manglar'),
    1816855553143980067: ('Infraestructura','Parque espíritu del manglar'),
    1816855553143980067: ('Deporte','Niñez')
}

def asignar_temas(tweet_id):
    if tweet_id in temas_generales: 
        return temas_generales[tweet_id]
    else: 
        ('No asignado', 'No asignado')

data[['Tema Principal', 'Tema Secundario']] = data['Tweet ID'].apply(lambda x: pd.Series(asignar_temas(x)))


In [38]:

# Guardar el DataFrame resultante en un archivo CSV
data.to_csv('F:\ManuelAngulo\PROYECTOS\ANALISIS DE SENTIMIENTOS\data\X_dumek_data_filtrado.csv', index=False, header=True)